In [38]:
import psycopg2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

import seaborn as sns
import datetime

##DataFrames To be used for analysis of outfielders
#Reguler_Outflder_Stats_30.csv   *played atleast 30 games
#Reguler_Outflder_Stats_100.csv  *played atleast 100 games
#All_Star_Outflder_Stats_100.csv  *alstars who played atleast 100 gms

In [39]:
import psycopg2 as p2
conn = p2.connect(database = 'player_stats', user ='baseball_master', password = 'georgetowndata', 
                  host = 'georgetown-baseball.cnfqonxqdbry.us-east-1.rds.amazonaws.com', 
                  port = '5432')

### Non- All Star Outfielders who have played at least 30 games at their position

In [40]:
query="""
select "playerID", "POS", "yearID", "GS", "G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'OF'
and "G" > 30 and "G" < 100
and "playerID" not in 
   (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

Reguler_Outfielder_30 = pd.read_sql_query(query, conn)

In [41]:
Reguler_Outfielder_30.head(3)

,playerID,POS,yearID,GS,G
0,aberscl01,OF,1947.0,NaN,40.0
1,adamsbu01,OF,1947.0,NaN,51.0
2,binksge01,OF,1947.0,NaN,75.0


## Batting Stats for Non All Stars Outfielders who played atleast 30 games

In [42]:
query="""
select a."playerID", m."nameFirst", m."nameLast", m."birthYear", m."deathYear", a."yearID", a."G", a."teamID", a."AB",a."H",a."2B", a."3B",
a."HR", a."SO", a."HBP", a."BB", a."SF"
from batting a
join master m
on a."playerID" = m."playerID"
where a."yearID" >= '1947'
"""

Batting_Stats = pd.read_sql_query(query,conn)

In [43]:
Batting_Stats['AVG'] = Batting_Stats['H'] / Batting_Stats['AB']

In [44]:
Batting_Stats["1B"] = Batting_Stats["H"] - Batting_Stats["2B"]- Batting_Stats["3B"]
- Batting_Stats["HR"]

0        -0.0
1        -4.0
2        -4.0
3        -2.0
4        -2.0
5        -2.0
6        -0.0
7        -8.0
8        -0.0
9        -0.0
10       -0.0
11       -0.0
12       -0.0
13       -0.0
14       -1.0
15       -0.0
16       -0.0
17       -0.0
18       -4.0
19       -1.0
20       -5.0
21       -0.0
22       -0.0
23       -0.0
24       -0.0
25       -0.0
26       -0.0
27       -0.0
28       -0.0
29       -0.0
         ... 
70052    -0.0
70053    -3.0
70054    -5.0
70055    -0.0
70056    -0.0
70057    -0.0
70058    -0.0
70059    -0.0
70060    -7.0
70061    -0.0
70062    -0.0
70063    -0.0
70064    -0.0
70065    -0.0
70066   -21.0
70067    -0.0
70068    -0.0
70069    -0.0
70070    -0.0
70071    -9.0
70072    -0.0
70073    -0.0
70074    -0.0
70075    -0.0
70076    -0.0
70077    -0.0
70078   -15.0
70079   -18.0
70080   -12.0
70081    -0.0
Name: HR, Length: 70082, dtype: float64

In [45]:
Batting_Stats.HBP.fillna(0, inplace = True)  #replace NaN with zero

In [46]:
Batting_Stats.SF.fillna(0, inplace = True)  #replace NaN with zero

In [47]:
Batting_Stats["TB"] = Batting_Stats["1B"] + Batting_Stats["2B"]*2 
+ Batting_Stats["3B"]*3 + Batting_Stats["HR"] * 4

0         0.0
1        25.0
2        22.0
3        11.0
4        17.0
5         8.0
6         0.0
7        32.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        9.0
14        4.0
15        0.0
16        0.0
17        0.0
18       22.0
19        4.0
20       47.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
70052     0.0
70053    18.0
70054    41.0
70055     0.0
70056     0.0
70057     0.0
70058     0.0
70059     0.0
70060    28.0
70061     0.0
70062     0.0
70063     0.0
70064     0.0
70065     0.0
70066    93.0
70067     0.0
70068     0.0
70069     0.0
70070     0.0
70071    36.0
70072     0.0
70073     0.0
70074     0.0
70075     0.0
70076     0.0
70077     0.0
70078    63.0
70079    81.0
70080    48.0
70081     0.0
Length: 70082, dtype: float64

In [48]:
Batting_Stats["SLG"] = Batting_Stats["TB"] / Batting_Stats["AB"]

In [49]:
Batting_Stats["OBP"] = (Batting_Stats["H"] + Batting_Stats["BB"] 
+ Batting_Stats["HBP"]) / (Batting_Stats ["AB"] + Batting_Stats ["BB"]
+ Batting_Stats["HBP"] + Batting_Stats["SF"])

In [50]:
Batting_Stats.head(3)

,playerID,nameFirst,nameLast,birthYear,deathYear,yearID,G,teamID,AB,H,...,HR,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP
0,abernwo01,Woody,Abernathy,1915.0,1994.0,1947.0,1.0,NY1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN
1,aberscl01,Cliff,Aberson,1921.0,1973.0,1947.0,47.0,CHN,140.0,39.0,...,4.0,32.0,0.0,20.0,0.0,0.278571,30.0,42.0,0.300000,0.368750
2,adamsbo03,Bobby,Adams,1921.0,1997.0,1947.0,81.0,CIN,217.0,59.0,...,4.0,23.0,4.0,25.0,0.0,0.271889,46.0,68.0,0.313364,0.357724


In [51]:
Batting_Stats["OPS"] = Batting_Stats["OBP"] + Batting_Stats["SLG"]

### Merge Outfielder Dataframe with Batting Avg Dataframe

In [52]:
Reguler_Outflder_Stats_30= pd.merge(Reguler_Outfielder_30, Batting_Stats, on = ["playerID", "yearID"])

In [53]:
Reguler_Outflder_Stats_30.head(3)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,aberscl01,OF,1947.0,NaN,40.0,Cliff,Aberson,1921.0,1973.0,47.0,...,32.0,0.0,20.0,0.0,0.278571,30.0,42.0,0.300000,0.368750,0.668750
1,adamsbu01,OF,1947.0,NaN,51.0,Buster,Adams,1915.0,1990.0,69.0,...,29.0,0.0,26.0,0.0,0.247253,33.0,55.0,0.302198,0.341346,0.643544
2,binksge01,OF,1947.0,NaN,75.0,George,Binks,1914.0,2010.0,104.0,...,36.0,1.0,23.0,0.0,0.258258,63.0,101.0,0.303303,0.308123,0.611427


In [54]:
Reguler_Outflder_Stats_30.to_csv('Reguler_Outflder_Stats_30.csv')

### Non- All Star Outfielders who have played at least 100 games at their position and Batting Stats

In [55]:
query="""

select "playerID", "POS", "yearID", "GS","G"
from fielding 
where "G" > 100
and "yearID" >= '1947'
and "POS" = 'OF'
and "playerID" not in 
	(Select "playerID"  ---count("playerID")
from allstar_full
group by "playerID"
having count("playerID") > 5)
"""

Reguler_Outflder_100 = pd.read_sql_query(query, conn)

In [56]:
Reguler_Outflder_Stats_100 = pd.merge(Reguler_Outflder_100, Batting_Stats, on = ["playerID", "yearID"])

In [57]:
Reguler_Outflder_Stats_100.head(3)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,baumhfr01,OF,1947.0,NaN,150.0,Frank,Baumholtz,1918.0,1997.0,151.0,...,53.0,1.0,56.0,0.0,0.283048,141.0,205.0,0.318818,0.341429,0.660247
1,chapmsa01,OF,1947.0,NaN,146.0,Sam,Chapman,1916.0,2006.0,149.0,...,70.0,0.0,65.0,0.0,0.252269,116.0,152.0,0.275862,0.331169,0.607031
2,ennisde01,OF,1947.0,NaN,135.0,Del,Ennis,1925.0,1996.0,139.0,...,51.0,3.0,37.0,0.0,0.275416,118.0,168.0,0.310536,0.325301,0.635837


In [58]:
Reguler_Outflder_Stats_100.to_csv('Reguler_Outflder_Stats_100.csv')

### All Star Infielders who have plaved in over 5 All Star games and over 100 games

In [59]:
query="""
select "playerID", "POS", "yearID", "GS", "G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'OF'
and "G" > 100
and "playerID" in  
        (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

All_Star_Outflder_100 = pd.read_sql_query(query, conn)

In [60]:
All_Star_Outflder_Stats_100 = pd.merge(All_Star_Outflder_100, Batting_Stats, on = ["playerID", "yearID"])

In [61]:
All_Star_Outflder_Stats_100.head(3)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,dimagdo01,OF,1947.0,NaN,134.0,Dom,DiMaggio,1917.0,2009.0,136.0,...,62.0,3.0,74.0,0.0,0.282651,119.0,161.0,0.313840,0.376271,0.690111
1,dimagjo01,OF,1947.0,NaN,139.0,Joe,DiMaggio,1914.0,1999.0,141.0,...,32.0,3.0,64.0,0.0,0.314607,127.0,189.0,0.353933,0.391015,0.744948
2,kinerra01,OF,1947.0,NaN,152.0,Ralph,Kiner,1922.0,2014.0,152.0,...,81.0,2.0,98.0,0.0,0.313274,150.0,196.0,0.346903,0.416541,0.763444


In [62]:
All_Star_Outflder_Stats_100.to_csv('All_Star_Outflder_Stats_100.csv')